In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot = True)


n_classes = 10
batch_size = 100

x = tf.placeholder('float', [None, 784])
y = tf.placeholder('float', [None, n_classes])


def conv2d(x,W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def maxpool2d(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')


def convolutional_neural_network(x):
    weights = {'W_conv1':tf.Variable(tf.random_normal([5, 5, 1, 32])), #32 filter 5x5 size
               'W_conv2':tf.Variable(tf.random_normal([5, 5, 32, 64])),#64 filter 5x5 size
               'W_fc':tf.Variable(tf.random_normal([7*7*64, 1024])),
               'out':tf.Variable(tf.random_normal([1024, n_classes])),
              }

    biases = {'b_conv1':tf.Variable(tf.random_normal([32])), #32 filter 5x5 size
               'b_conv2':tf.Variable(tf.random_normal([64])),#64 filter 5x5 size
               'b_fc':tf.Variable(tf.random_normal([1024])),
               'out':tf.Variable(tf.random_normal([n_classes])),
              }
    
    x= tf.reshape(x, [-1, 28, 28, 1])
    
    #Conv layer 1
    conv1 =  tf.nn.relu(conv2d(x, weights['W_conv1']))
    conv1 = maxpool2d(conv1)
    
    #Conv layer 2
    conv2 =  tf.nn.relu(conv2d(conv1, weights['W_conv2']))
    conv2 = maxpool2d(conv2)
    
    #FC 
    fc =tf.reshape(conv2, [-1, 7*7*64])
    fc= tf.nn.relu(tf.matmul(fc, weights["W_fc"]) +biases['b_fc'])
    
    output =tf.matmul(fc, weights["out"]) +biases['out']
    
    return output
  


Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [0]:
def train_neural_network(x):
    prediction = convolutional_neural_network(x)
    # OLD VERSION:
    #cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(prediction,y) )
    # NEW:
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y) )
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    hm_epochs = 10
    with tf.Session() as sess:
        # OLD:
        #sess.run(tf.initialize_all_variables())
        # NEW:
        sess.run(tf.global_variables_initializer())

        for epoch in range(hm_epochs):
            epoch_loss = 0
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x, epoch_y = mnist.train.next_batch(batch_size)
                _, c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
                epoch_loss += c

            print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)

        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))

        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:',accuracy.eval({x:mnist.test.images, y:mnist.test.labels}))


In [3]:
train_neural_network(x)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Epoch 0 completed out of 10 loss: 1379648.5431938171
Epoch 1 completed out of 10 loss: 266900.5826268196
Epoch 2 completed out of 10 loss: 154480.89444863796
Epoch 3 completed out of 10 loss: 102173.27388811111
Epoch 4 completed out of 10 loss: 75533.72664892673
Epoch 5 completed out of 10 loss: 52039.205483317375
Epoch 6 completed out of 10 loss: 36777.60465455055
Epoch 7 completed out of 10 loss: 28290.3861194849
Epoch 8 completed out of 10 loss: 23154.28645281733
Epoch 9 completed out of 10 loss: 19989.68257600069
Accuracy: 0.977


In [4]:
mnist

Datasets(train=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7fbc0ab01828>, validation=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7fbc033ebcc0>, test=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7fbc033eba90>)